# LLM Society: End-to-End Tutorial (English)

This notebook provides a practical, end-to-end tour of the `llm-society` package:
- Installation and API key setup
- Quick start (simple mode, no LLM calls)
- LLM mode with conversations and belief updates
- Visualizations via `net.plot(type=...)`
- Persona segments
- Scheduled intervention on selected nodes
- Using a custom NetworkX graph (e.g., real-world data)
- Conversation retrieval, summary panel, and exporting results
- CLI usage cheatsheet

Notes:
- All information strings and comments are in English for portability.
- LLM mode requires a valid OpenAI API key (see below).



## Installation and setup

```bash
pip install llm-society
```

Set your OpenAI API key (only needed for LLM mode):
```bash
export OPENAI_API_KEY=<YOUR_OPENAI_API_KEY>
# Or save to file (gitignored by default)
echo "<YOUR_OPENAI_API_KEY>" > api-key.txt
```
The library reads from `OPENAI_API_KEY` or `OPENAI_API_KEY_FILE`.



In [ ]:
# Imports
from llm_society import network
import matplotlib.pyplot as plt



## Quick start: simple mode (no LLM)
This mode is fast, deterministic given a seed, and ideal for demos without API calls.



In [ ]:
net_simple = network(
    information="Increasing the carbon tax will reduce emissions.",
    n=12, degree=3, rounds=6,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.9,
    mode="simple", rng=0,
)
net_simple.simulate()
net_simple.plot(type="coverage")
net_simple.plot(type="final_beliefs")



## LLM mode: conversations and belief updates
Requires an OpenAI API key. This mode generates short conversations and updates beliefs.



In [ ]:
net_llm = network(
    information="Increasing gas taxes will significantly reduce inflation.",
    n=20, degree=4, rounds=8,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.95, talk_prob=0.5,
    mode="llm", rng=0, model="gpt-4.1",
)
net_llm.simulate()
# Optional animation (may be slow)
# net_llm.plot(type="animation", save="anim.gif")
net_llm.plot(type="coverage")



## Visualizations via `net.plot(type=...)`
Available types: `animation`, `coverage`, `final_beliefs`, `group_beliefs`, `centrality`, `intervention_effect`.



In [ ]:
# Group mean beliefs (by persona attribute)
net_llm.plot(type="group_beliefs", attr="political")

# Centrality vs final belief/exposure (try: degree|betweenness|eigenvector)
net_llm.plot(type="centrality", metric="betweenness")



## Persona segments
Define sub-populations to induce structured heterogeneity (e.g., political alignment).



In [ ]:
segments = [
    {"proportion": 0.5, "traits": {"political": {"choices": {"Democrat": 0.85, "Republican": 0.15}}}},
    {"proportion": 0.5, "traits": {"political": {"choices": {"Democrat": 0.15, "Republican": 0.85}}}},
]

net_seg = network(
    information="Increasing the minimum wage will reduce inequality.",
    n=24, degree=4, rounds=8,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.95, talk_prob=0.5,
    mode="llm", rng=0, model="gpt-4.1",
    segments=segments,
)
net_seg.simulate()
net_seg.plot(type="group_beliefs", attr="political")



## Scheduled intervention on selected nodes
Reduce the probability of discussing the target information for edges touching selected nodes starting at a given round.



In [ ]:
net_int = network(
    information="Expanding vocational training will increase employment.",
    n=24, degree=4, rounds=10,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.95, talk_prob=0.5,
    mode="llm", rng=0, model="gpt-4.1",
    intervention_round=6,
    intervention_nodes=[2,5,7,11,13,17],
    intervention_talk_prob_multiplier=0.5,
)
net_int.simulate()
net_int.plot(type="intervention_effect", intervention_round=6)



## Using a custom NetworkX graph (e.g., from real data)
Nodes will be relabeled to 0..N-1 if not already. Missing edge weights are auto-filled.



In [ ]:
import networkx as nx

G = nx.karate_club_graph()
net_custom = network(
    information="Raising the minimum wage will rapidly reduce unemployment.",
    degree=4, rounds=8,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.95, talk_prob=0.5,
    mode="llm", rng=1, model="gpt-4.1",
    graph=G,
)
net_custom.simulate()
net_custom.plot(type="coverage")



## Conversation retrieval, summary panel, and exporting results
All per-edge conversations are stored per round in `history[t]["conversations"]` in LLM mode.



In [ ]:
# Use the previously simulated net_llm
# All conversations at round 5 (list of records)
convos_r5 = net_llm.conversations(round=5)
print(f"Round 5 conversations: {len(convos_r5)} edges")
if convos_r5:
    print(convos_r5[0].keys())

# Specific conversation between nodes 2 and 7 at round 5
rec_2_7 = net_llm.get_conversation(round=5, u=2, v=7)
if rec_2_7:
    print("Did talk:", rec_2_7["did_talk"]) 
    for line in rec_2_7["turns"][:6]:
        print(line)
else:
    print("No record for (2,7) at round 5")

# Quick summary and exports
print(net_llm.summary())
net_llm.export(
    history_csv="history_tutorial.csv",
    beliefs_csv="beliefs_tutorial.csv",
    conversations_jsonl="conversations_tutorial.jsonl",
)
print("Exported history_tutorial.csv, beliefs_tutorial.csv, conversations_tutorial.jsonl")



## CLI usage cheatsheet

Write an example config:
```bash
llm-society --write-example-config my-config.yaml
```
Run a simulation from config:
```bash
llm-society --config my-config.yaml
```
Override parameters via flags:
```bash
llm-society --config my-config.yaml --depth 0.8 --rounds 20
```
Run fully via flags (with intervention):
```bash
llm-society \
  --information "Increasing gas taxes will significantly reduce inflation." \
  --n 24 --degree 4 --rounds 10 \
  --depth 0.5 --depth-max 4 --edge-frac 0.5 \
  --seeds 0,1 --seed-belief 0.95 --talk-prob 0.5 \
  --mode llm --rng 0 --model gpt-4.1 \
  --intervention-round 6 --intervention-nodes 2,5,7,11,13,17 --intervention-multiplier 0.5
```

